### Simulaciones

Estos son los parámetros que le pasaremos a nuestos agentes para crear las simulaciones, los siguientes parametros tienen valores aleatorios que usamos en la mayoria de las simulaciones, sin embargo, luego los modificamos para observar ciertos comportamientos.

In [1]:
import random
from PMOntologic.Opportunity import *
from PMOntologic.PMO import Project
from PMOntologic.Resource import Resource
from Simulation.Simulator import Simulator
from Simulation.Environment import WorkCenter
from Simulation.PMAgent import *
from Simulation.WorkerAgent import WorkerAgent
from Tasks.GeneticAlgorithm.Population import Population
from Tasks.GeneticAlgorithm.Tasks_combination import optimization_function
from Tasks.task import Task
from PMOntologic.Risk import *

# Definimos los parametros de los agentes trabajadores
MIN_PS_AGENTS = 10          # Numero minimo de resolucion de problemas de los agentes
MAX_PS_AGENTS = 100         # Numero maximo de resolucion de problemas de los agentes
MIN_FRIENDSHIP = 0.0        # Minimo de amistad entre los agentes
MAX_FRIENDSHIP = 1.0        # Maximo de amistad entre los agentes
MIN_MIN_ENERGY = 10         # Minima energia de los agentes para trabajar
MAX_MIN_ENERGY = 100        # Maxima energia de los agentes para trabajar
MIN_MOTIVATION = 10         # Motivacion minima para que los trabajadores se sientan motivados
MIN_LAZZY = 0.0             # Pereza minima de los agentes
MAX_LAZZY = 0.4             # Pereza maxima de los agentes 

# Definimos los parametros del Project Manager
RISKY = 0.2                 # Cuan arriesgado es el project manager
MIN_WORK_PROB = 0           # Probabilidad minima de que elija trabajar él antes de mandar a un trabajador
MAX_WORK_PROB = 0.2  
MIN_COOPERATION = 0.5       # Probabilidad minima de que elija mandar a cooperar en tareas dificiles, 1 - P seria la probabilidad de que reasigne en vez de cooperar
MAX_COOPERATION = 1.0
MIN_EXPLORATION = 0.0       # Probabilidad minima de que tome decisiones poco usuales
MAX_EXPLORATION = 0.2 

# Definimos algunos parametros para las tareas
MIN_DURATION = 5            # Duracion minima de una tarea
MAX_DURATION = 100         
INTERVAL = 1.2              # Porcentaje de tiempo entre el start y el deadline de la tarea
MIN_RESOURCES = 1           # Cantidad minima de recursos que tiene una tarea
MAX_RESOURCES = 10
MIN_DEPENDENCIES = 0        # Cantidad minima de dependencias que tiene una tarea
MAX_DEPENDENCIES = 10
MIN_REWARD = 1              # Recompensa minima que debe tener una tarea
MAX_REWARD = 100 
MIN_DIFFICULTY = 1          # Dificultad minima que debe tener una tarea
MAX_DIFFICULTY = 100
MIN_PROBLEMS_PROB = 0       # Probabilidad minima de una tarea de generar un problema
MAX_PROBLEMS_PROB = 0.5     

NUMBER_OF_TASKS = 50
NUMBER_OF_WORKERS = 10


##### Definimos diferentes proyectos sobre los que haremos nuestras simulaciones

In [2]:
resources_id = ['budget','Resource1', 'Resource2', 'Resource3', 'Resource4', 'Resource5', 'Resource6', 'Resource7', 'Resource8', 'Resource9', 'Resource10']
resources_cant = {resource : 0 for resource in resources_id}

risks = [ExecutionDelayRisk(), AcceleratedSpendingRisk(), CostIncreaseRisk(), UnfulfilledDependencesRisk(), LowProductivityRisk(), LowMotivationRisk(), LowPriorityRisk()]
opportunities = [HighProductivityOpportunity(), HighMotivationOpportunity(), OptimizedResourcesOpportunity(), CostSavingOpportunity(), EarlyCompletionOpportunity(), HighCooperationOpportunity(), MilestoneCompletionOpportunity(), RiskManagementSuccessOpportunity(), InnovationOpportunity()]

# Definimos varios conjuntos de tareas para usarlos posteriormente
tasks_params = [
    { #Conjunto de tareas problemáticas y difíciles
        "MIN_DURATION": 50,            # Tareas de mayor duración
        "MAX_DURATION": 100,
        "INTERVAL": 1.5,               # Intervalo de tiempo más ajustado
        "MIN_RESOURCES": 5,            # Requieren más recursos
        "MAX_RESOURCES": 10,
        "MIN_DEPENDENCIES": 3,         # Más dependencias
        "MAX_DEPENDENCIES": 10,
        "MIN_REWARD": 50,              # Recompensas más altas
        "MAX_REWARD": 100,
        "MIN_DIFFICULTY": 70,          # Dificultad alta
        "MAX_DIFFICULTY": 100,
        "MIN_PROBLEMS_PROB": 0.3,      # Alta probabilidad de problemas
        "MAX_PROBLEMS_PROB": 0.5
    }, { # Conjunto de tareas fáciles y sin problemas
        "MIN_DURATION": 5,             # Tareas cortas
        "MAX_DURATION": 30,
        "INTERVAL": 2.0,               # Más tiempo antes del deadline
        "MIN_RESOURCES": 1,            # Requieren pocos recursos
        "MAX_RESOURCES": 3,
        "MIN_DEPENDENCIES": 0,         # Sin dependencias
        "MAX_DEPENDENCIES": 2,
        "MIN_REWARD": 1,               # Recompensa baja
        "MAX_REWARD": 20,
        "MIN_DIFFICULTY": 1,           # Dificultad baja
        "MAX_DIFFICULTY": 30,
        "MIN_PROBLEMS_PROB": 0,        # Casi sin problemas
        "MAX_PROBLEMS_PROB": 0.1
    }, { # Conjunto de tareas complejas pero recompensadas
        "MIN_DURATION": 20,            # Tareas de duración media
        "MAX_DURATION": 80,
        "INTERVAL": 1.4,               # Intervalo ajustado
        "MIN_RESOURCES": 3,            # Requieren más recursos
        "MAX_RESOURCES": 8,
        "MIN_DEPENDENCIES": 2,         # Varias dependencias
        "MAX_DEPENDENCIES": 7,
        "MIN_REWARD": 70,              # Alta recompensa
        "MAX_REWARD": 100,
        "MIN_DIFFICULTY": 50,          # Dificultad media-alta
        "MAX_DIFFICULTY": 90,
        "MIN_PROBLEMS_PROB": 0.1,      # Media probabilidad de problemas
        "MAX_PROBLEMS_PROB": 0.4
    }, { # Conjunto de tareas urgentes y dependientes
        "MIN_DURATION": 10,            # Duración corta
        "MAX_DURATION": 40,
        "INTERVAL": 1.1,               # Poco tiempo antes del deadline
        "MIN_RESOURCES": 4,            # Requieren varios recursos
        "MAX_RESOURCES": 7,
        "MIN_DEPENDENCIES": 4,         # Muchas dependencias
        "MAX_DEPENDENCIES": 10,
        "MIN_REWARD": 30,              # Recompensa media
        "MAX_REWARD": 70,
        "MIN_DIFFICULTY": 60,          # Dificultad alta
        "MAX_DIFFICULTY": 90,
        "MIN_PROBLEMS_PROB": 0.2,      # Probabilidad media de problemas
        "MAX_PROBLEMS_PROB": 0.4
    }, { # Conjunto de tareas sencillas con baja recompensa
        "MIN_DURATION": 5,             # Tareas muy cortas
        "MAX_DURATION": 20,
        "INTERVAL": 1.8,               # Tiempo relajado para completarlas
        "MIN_RESOURCES": 1,            # Pocos recursos
        "MAX_RESOURCES": 4,
        "MIN_DEPENDENCIES": 0,         # Sin dependencias
        "MAX_DEPENDENCIES": 3,
        "MIN_REWARD": 5,               # Baja recompensa
        "MAX_REWARD": 20,
        "MIN_DIFFICULTY": 1,           # Dificultad muy baja
        "MAX_DIFFICULTY": 40,
        "MIN_PROBLEMS_PROB": 0,        # Sin problemas
        "MAX_PROBLEMS_PROB": 0.2
    }, { #  Conjunto de tareas de alto riesgo y recompensa
        "MIN_DURATION": 30,            # Tareas largas
        "MAX_DURATION": 100,
        "INTERVAL": 1.3,               # Intervalo ajustado
        "MIN_RESOURCES": 5,            # Muchos recursos necesarios
        "MAX_RESOURCES": 10,
        "MIN_DEPENDENCIES": 3,         # Varias dependencias
        "MAX_DEPENDENCIES": 8,
        "MIN_REWARD": 80,              # Recompensa alta
        "MAX_REWARD": 100,
        "MIN_DIFFICULTY": 70,          # Dificultad alta
        "MAX_DIFFICULTY": 100,
        "MIN_PROBLEMS_PROB": 0.4,      # Alta probabilidad de problemas
        "MAX_PROBLEMS_PROB": 0.5
    }
]
tasks_sets = []
projects = []
for s in range(len(tasks_params)):
    task_set = []
    for i in range(NUMBER_OF_TASKS):
        task = Task(id=i, priority=random.randint(0,5), duration=random.randint(tasks_params[s]["MIN_DURATION"], tasks_params[s]["MAX_DURATION"]), reward=random.randint(tasks_params[s]["MIN_REWARD"],tasks_params[s]["MAX_REWARD"]), difficulty=random.randint(tasks_params[s]["MIN_DIFFICULTY"],tasks_params[s]["MAX_DIFFICULTY"]), problems_probability=random.uniform(tasks_params[s]["MIN_PROBLEMS_PROB"],tasks_params[s]["MAX_PROBLEMS_PROB"]))
        cant = random.randint(0,10)
        for _ in range(cant):
            r = random.choice(resources_id)
            amount = random.randint(0,5)
            resource = Resource(r, amount)
            task.resources.append(resource)
            resources_cant[r] += amount 
        task_set.append(task)

    for i in range(NUMBER_OF_TASKS - 1):
        if random.random() < 0.2 :
            dep = random.randint(int(tasks_params[s]["MIN_DEPENDENCIES"]),int(tasks_params[s]["MAX_DEPENDENCIES"]))
            for j in range(dep):
                index = random.randint(i+1, NUMBER_OF_TASKS - 1)
                task_set[i].dependencies.append(task_set[index])

    tasks_sets.append(task_set)
    projects.append(Project(objective='number_of_tasks',tasks=task_set, resources=[Resource(resource,cant*1.2) for resource,cant in resources_cant.items()], risks=risks, opportunities=opportunities))


simulator = Simulator(WorkCenter)
simulations = []

rules = {'ReduceTime': ReduceTimeRule(), 'OptimizeResource': OptimizeResourceRule(),
        'AvoidRisks': AvoidRisksRule(), 'TakeOpportunities': TakeOpportunitiesRule(), 
        'KeepMotivation': KeepMotivationRule(), 'ChangeTarget': ChangeTargetRule(), 
        'Assign': AssignRule(), 'Problems': ProblemsRule(), 'ForgetRisks':ForgetRisksRule(),
        'DeleteActive' : DeleteActiveRule()}

active_rules = ['ReduceTime', 'OptimizeResource', 'AvoidRisks', 'TakeOpportunities', 'KeepMotivation', 'ChangeTarget', 'Assign', 'Problems', 'ForgetRisks', 'DeleteActive' ]



##### Definimos varios Project Manager con diferentes parámetros

In [3]:
# Definimos los parametros del Project Manager
PMs_params = [
    { # Altamente arriesgado
    "RISKY" : 0.7,               
    "MIN_WORK_PROB" : 0,         
    "MAX_WORK_PROB" : 0.2,
    "MIN_COOPERATION" : 0,    
    "MAX_COOPERATION" : 1,
    "MIN_EXPLORATION" : 0.3,  
    "MAX_EXPLORATION" : 0.7 
    }, { # Poco arriesgado
    "RISKY" : 0.2,               
    "MIN_WORK_PROB" : 0,         
    "MAX_WORK_PROB" : 0.2,
    "MIN_COOPERATION" : 0,    
    "MAX_COOPERATION" : 1,
    "MIN_EXPLORATION" : 0,  
    "MAX_EXPLORATION" : 0.2 
    }, { # Trabajador
    "RISKY" : 0.2,               
    "MIN_WORK_PROB" : 0.3,         
    "MAX_WORK_PROB" : 0.7,
    "MIN_COOPERATION" : 0.5,    
    "MAX_COOPERATION" : 1,
    "MIN_EXPLORATION" : 0,  
    "MAX_EXPLORATION" : 0.2 
    }, { # Cooperativo
    "RISKY" : 0.2,               
    "MIN_WORK_PROB" : 0,         
    "MAX_WORK_PROB" : 0.2,
    "MIN_COOPERATION" : 0.7,    
    "MAX_COOPERATION" : 1,
    "MIN_EXPLORATION" : 0,  
    "MAX_EXPLORATION" : 0.2 
    }, { # Trabajador, Cooperativo y Arriesgado
    "RISKY" : 0.7,               
    "MIN_WORK_PROB" : 0.3,         
    "MAX_WORK_PROB" : 0.7,
    "MIN_COOPERATION" : 0.7,    
    "MAX_COOPERATION" : 1,
    "MIN_EXPLORATION" : 0,  
    "MAX_EXPLORATION" : 0.2 
    }
    ]

PMs = []
# Iniciamos por cada proyecto varios PMs con diferentes características
for project in projects:
    last = PMAgent(min_motivation_team=20, initial_perception=PMperception(actual_time=0, team_motivation=80, reward=0), project=project, rules=rules, active_rules=active_rules, risky=PMs_params[len(PMs_params)-1]["RISKY"], work_prob=random.uniform(PMs_params[len(PMs_params)-1]["MIN_WORK_PROB"], PMs_params[len(PMs_params)-1]["MAX_WORK_PROB"]), cooperation_prob=random.uniform(PMs_params[len(PMs_params)-1]["MIN_COOPERATION"], PMs_params[len(PMs_params)-1]["MAX_COOPERATION"]), exploration_rate=random.uniform(PMs_params[len(PMs_params)-1]["MIN_EXPLORATION"], PMs_params[len(PMs_params)-1]["MAX_EXPLORATION"]))
    for i in range(len(PMs_params)-1):
        PMs.append(PMAgent(min_motivation_team=20, initial_perception=PMperception(actual_time=0, team_motivation=80, reward=0), ordered_tasks=last.ordered_tasks, project=project, rules=rules, active_rules=active_rules, risky=PMs_params[i]["RISKY"], work_prob=random.uniform(PMs_params[i]["MIN_WORK_PROB"], PMs_params[i]["MAX_WORK_PROB"]), cooperation_prob=random.uniform(PMs_params[i]["MIN_COOPERATION"], PMs_params[i]["MAX_COOPERATION"]), exploration_rate=random.uniform(PMs_params[i]["MIN_EXPLORATION"], PMs_params[i]["MAX_EXPLORATION"])))
    PMs.append(last)

##### Se realizaron varias simulaciones para intentar responder las siguientes interrogantes

¿Cómo afecta el tamaño del equipo al número de tareas completadas en un período de tiempo?

Para ello simulamos varios equipos pequeños (de 4 trabajadores) y varios equipos grandes (de 10 trabajadores) resolviendo los mismos proyectos.

In [4]:
from copy import deepcopy

agents_big_team = []

# Definimos los parametros de los agentes trabajadores
MIN_PS_AGENTS = 30          # Numero minimo de resolucion de problemas de los agentes
MAX_PS_AGENTS = 50          # Numero maximo de resolucion de problemas de los agentes
MIN_FRIENDSHIP = 0.2        # Minimo de amistad entre los agentes
MAX_FRIENDSHIP = 0.5        # Maximo de amistad entre los agentes
MIN_MIN_ENERGY = 10         # Minima energia de los agentes para trabajar
MAX_MIN_ENERGY = 100        # Maxima energia de los agentes para trabajar
MIN_MOTIVATION = 10         # Motivacion minima para que los trabajadores se sientan motivados
MIN_LAZZY = 0.0             # Pereza minima de los agentes
MAX_LAZZY = 0.4             # Pereza maxima de los agentes 

NUMBER_OF_WORKERS = 10 


for i in range(NUMBER_OF_WORKERS):
    agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(MIN_PS_AGENTS,MAX_PS_AGENTS), friendship=random.uniform(MIN_FRIENDSHIP,MAX_FRIENDSHIP),lazzy=random.uniform(MIN_LAZZY,MAX_LAZZY), max_energy=MAX_MIN_ENERGY, min_motivation=MIN_MOTIVATION, min_energy=MIN_MIN_ENERGY)
    agents_big_team.append(agent)

for pm in PMs:
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_big_team])
    project_ = deepcopy(pm_.project)
    simulations.append(simulator.StartSimulation(1000,agents_big_team,pm_,project_))


agents_small_team = []

# Definimos los parametros de los agentes trabajadores
MIN_PS_AGENTS = 30          
MAX_PS_AGENTS = 50          
MIN_FRIENDSHIP = 0.5        # Aumentamos la amistad entre los trabajadores ya que equipos pequeños tienden a ser
MAX_FRIENDSHIP = 0.8        # mas unidos que equipos grandes.
MIN_MIN_ENERGY = 10         
MAX_MIN_ENERGY = 100        
MIN_MOTIVATION = 10         
MIN_LAZZY = 0.0             
MAX_LAZZY = 0.4              

NUMBER_OF_WORKERS = 4


for i in range(NUMBER_OF_WORKERS):
    agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(MIN_PS_AGENTS,MAX_PS_AGENTS), friendship=random.uniform(MIN_FRIENDSHIP,MAX_FRIENDSHIP), lazzy=random.uniform(MIN_LAZZY,MAX_LAZZY), max_energy=MAX_MIN_ENERGY, min_motivation=MIN_MOTIVATION, min_energy=MIN_MIN_ENERGY)
    agents_small_team.append(agent)

for pm in PMs:
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_small_team])
    project_ = deepcopy(pm_.project)
    simulations.append(simulator.StartSimulation(1000,agents_small_team,pm_,project_))


¿Cuál es el impacto de la motivación del equipo en la productividad general?

Para ello hicimos simulaciones con dos equipos, uno con nivel de motivación mínima de 10 y otro de 30. El nivel mínimo de motivación influye en el deseo de trabajar de los agentes.

In [5]:
agents_high_motivation = []
agents_low_motivation = []

# Definimos los parametros de los agentes trabajadores
MIN_PS_AGENTS = 30          
MAX_PS_AGENTS = 50          
MIN_FRIENDSHIP = 0.3        
MAX_FRIENDSHIP = 0.8        
MIN_MIN_ENERGY = 10         
MAX_MIN_ENERGY = 100                
MIN_LAZZY = 0.0             
MAX_LAZZY = 0.4              

NUMBER_OF_WORKERS = 10

# Simulaciones con agentes cuya motivacion puede disminuir hasta 10 sin dejar de trabajar
MIN_MOTIVATION = 10 
for i in range(NUMBER_OF_WORKERS):
    agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(MIN_PS_AGENTS,MAX_PS_AGENTS), friendship=random.uniform(MIN_FRIENDSHIP,MAX_FRIENDSHIP), lazzy=random.uniform(MIN_LAZZY,MAX_LAZZY), max_energy=MAX_MIN_ENERGY, min_motivation=MIN_MOTIVATION, min_energy=MIN_MIN_ENERGY)
    agents_low_motivation.append(agent)


for pm in PMs:
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_low_motivation])
    project_ = deepcopy(pm_.project)
    simulations.append(simulator.StartSimulation(1000,agents_low_motivation,pm_,project_))


# Simulaciones con agentes cuya motivacion puede disminuir hasta 30 sin dejar de trabajar
MIN_MOTIVATION = 30 
for i in range(NUMBER_OF_WORKERS):
    agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(MIN_PS_AGENTS,MAX_PS_AGENTS), friendship=random.uniform(MIN_FRIENDSHIP,MAX_FRIENDSHIP), lazzy=random.uniform(MIN_LAZZY,MAX_LAZZY), max_energy=MAX_MIN_ENERGY, min_motivation=MIN_MOTIVATION, min_energy=MIN_MIN_ENERGY)
    agents_high_motivation.append(agent)


for pm in PMs:
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_high_motivation])
    project_ = deepcopy(pm_.project)
    simulations.append(simulator.StartSimulation(1000,agents_high_motivation,pm_,project_))

¿Cómo influye la capacidad de resolución de problemas de los trabajadores en el éxito general del proyecto?

Igualmente haremos dos equipos, uno con poca preparación y otro con suficiente, simularemos varios escenarios de proyectos.

In [6]:
agents_high_ps = []
agents_low_ps = []

# Definimos los parametros de los agentes trabajadores       
MIN_FRIENDSHIP = 0.3        
MAX_FRIENDSHIP = 0.8        
MIN_MIN_ENERGY = 10         
MAX_MIN_ENERGY = 100   
MIN_MOTIVATION = 10              
MIN_LAZZY = 0.0             
MAX_LAZZY = 0.4              

NUMBER_OF_WORKERS = 10

# Simulaciones con agentes con poca preparacion
MIN_PS_AGENTS = 10          
MAX_PS_AGENTS = 40   
for i in range(NUMBER_OF_WORKERS):
    agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(MIN_PS_AGENTS,MAX_PS_AGENTS), friendship=random.uniform(MIN_FRIENDSHIP,MAX_FRIENDSHIP), lazzy=random.uniform(MIN_LAZZY,MAX_LAZZY), max_energy=MAX_MIN_ENERGY, min_motivation=MIN_MOTIVATION, min_energy=MIN_MIN_ENERGY)
    agents_low_ps.append(agent)


for pm in PMs:
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_low_ps])
    project_ = deepcopy(pm_.project)
    simulations.append(simulator.StartSimulation(1000,agents_low_ps,pm_,project_))


# Simulaciones con agentes con suficiente preparacion
MIN_PS_AGENTS = 40          
MAX_PS_AGENTS = 70  
for i in range(NUMBER_OF_WORKERS):
    agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(MIN_PS_AGENTS,MAX_PS_AGENTS), friendship=random.uniform(MIN_FRIENDSHIP,MAX_FRIENDSHIP), lazzy=random.uniform(MIN_LAZZY,MAX_LAZZY), max_energy=MAX_MIN_ENERGY, min_motivation=MIN_MOTIVATION, min_energy=MIN_MIN_ENERGY)
    agents_high_ps.append(agent)


for pm in PMs:
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_high_ps])
    project_ = deepcopy(pm_.project)
    simulations.append(simulator.StartSimulation(1000,agents_high_ps,pm_,project_))

¿Cómo afecta la colaboración entre trabajadores en la finalización de tareas complejas?

Esta pregunta puede empezar a ser respondida con la simulaciones de equipos grandes contra equipos pequeños, ahora pondremos al mismo equipo a enfrentarse a diferentes situaciones, solo variaremos la colaboración entre los agentes.

In [7]:
agents_high_col = []
agents_low_col = []

# Definimos los parametros de los agentes trabajadores     
MIN_PS_AGENTS = 20          
MAX_PS_AGENTS = 70    
MIN_MIN_ENERGY = 10         
MAX_MIN_ENERGY = 100   
MIN_MOTIVATION = 10              
MIN_LAZZY = 0.0             
MAX_LAZZY = 0.4              

NUMBER_OF_WORKERS = 10

# Simulaciones con agentes poco colaborativos
MIN_FRIENDSHIP = 0.05        
MAX_FRIENDSHIP = 0.5       
for i in range(NUMBER_OF_WORKERS):
    agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(MIN_PS_AGENTS,MAX_PS_AGENTS), friendship=random.uniform(MIN_FRIENDSHIP,MAX_FRIENDSHIP), lazzy=random.uniform(MIN_LAZZY,MAX_LAZZY), max_energy=MAX_MIN_ENERGY, min_motivation=MIN_MOTIVATION, min_energy=MIN_MIN_ENERGY)
    agents_low_col.append(agent)


for pm in PMs:
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_low_col])
    project_ = deepcopy(pm_.project)
    simulations.append(simulator.StartSimulation(1000,agents_low_col,pm_,project_))


# Simulaciones con agentes bastante colaborativos
MIN_FRIENDSHIP = 0.5        
MAX_FRIENDSHIP = 1 
for i in range(NUMBER_OF_WORKERS):
    agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(MIN_PS_AGENTS,MAX_PS_AGENTS), friendship=random.uniform(MIN_FRIENDSHIP,MAX_FRIENDSHIP), lazzy=random.uniform(MIN_LAZZY,MAX_LAZZY), max_energy=MAX_MIN_ENERGY, min_motivation=MIN_MOTIVATION, min_energy=MIN_MIN_ENERGY)
    agents_high_col.append(agent)


for pm in PMs:
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_high_col])
    project_ = deepcopy(pm_.project)
    simulations.append(simulator.StartSimulation(1000,agents_high_col,pm_,project_))

¿Qué sucede si el project manager promueve más colaboración entre trabajadores o reduce la misma?

Para responder a esta pregunta haremos un proyecto largo, con gran cantidad de tareas y de problemas, para ver cómo se ajusta el PM a las características del equipo que está manejando a medida que va interactuando con los agentes, dado que el PM no tiene información previa del equipo. También hicimos simulaciones donde le pasamos esta información previa al PM, pasandole un valor de cooperación acorde al equipo.

In [8]:
# Definimos algunos parametros para las tareas
MIN_DURATION = 5            
MAX_DURATION = 100         
INTERVAL = 1.2             
MIN_RESOURCES = 1          
MAX_RESOURCES = 10
MIN_DEPENDENCIES = 0        
MAX_DEPENDENCIES = 10
MIN_REWARD = 1             
MAX_REWARD = 100 
MIN_DIFFICULTY = 30          
MAX_DIFFICULTY = 70
MIN_PROBLEMS_PROB = 0.3       
MAX_PROBLEMS_PROB = 0.7     

NUMBER_OF_TASKS = 100
NUMBER_OF_WORKERS = 5

task_set = []
for i in range(NUMBER_OF_TASKS):
    task = Task(id=i, priority=random.randint(0,5), duration=random.randint(MIN_DURATION, MAX_DURATION), reward=random.randint(MIN_REWARD,MAX_REWARD), difficulty=random.randint(MIN_DIFFICULTY,MAX_DIFFICULTY), problems_probability=random.uniform(MIN_PROBLEMS_PROB,MAX_PROBLEMS_PROB))
    cant = random.randint(0,10)
    for _ in range(cant):
        r = random.choice(resources_id)
        amount = random.randint(0,5)
        resource = Resource(r, amount)
        task.resources.append(resource)
        resources_cant[r] += amount 
    task_set.append(task)

for i in range(NUMBER_OF_TASKS - 1):
    if random.random() < 0.2 :
        dep = random.randint(int(MIN_DEPENDENCIES),int(MAX_DEPENDENCIES))
        for j in range(dep):
            index = random.randint(i+1, NUMBER_OF_TASKS - 1)
            task_set[i].dependencies.append(task_set[index])

    
long_project = Project(objective='number_of_tasks',tasks=task_set, resources=[Resource(resource,cant*1.2) for resource,cant in resources_cant.items()], risks=risks, opportunities=opportunities)

pm = PMAgent(min_motivation_team=20, initial_perception=PMperception(actual_time=0, team_motivation=80, reward=0), project=long_project, rules=rules, active_rules=active_rules, risky=RISKY, work_prob=0.1, cooperation_prob=0.5, exploration_rate=0.1)

# Simulaciones con el equipo colaborativo y PM sin conocimiento
for _ in range(10):
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_high_col])
    project_ = deepcopy(long_project)
    simulations.append(simulator.StartSimulation(1000,agents_high_col,pm_,project_))

# Simulaciones con el equipo poco colaborativo y PM sin conocimiento
for _ in range(10):
    pm_ = deepcopy(pm)
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_low_col])
    project_ = deepcopy(long_project)
    simulations.append(simulator.StartSimulation(1000,agents_low_col,pm_,project_))

# Ahora le incorporamos el conocimiento del equipo
for _ in range(10):
    pm_ = deepcopy(pm)
    pm_.beliefs['cooperation_prob'] = 0.9
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_high_col])
    project_ = deepcopy(long_project)
    simulations.append(simulator.StartSimulation(1000,agents_high_col,pm_,project_))


for _ in range(10):
    pm_ = deepcopy(pm)
    pm_.beliefs['cooperation_prob'] = 0.1
    pm_.work_prob = 0.5
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents_low_col])
    project_ = deepcopy(long_project)
    simulations.append(simulator.StartSimulation(1000,agents_low_col,pm_,project_))

¿Cómo se desempeña el equipo bajo condiciones de baja motivación o baja cooperación?

Con las simulaciones hechas anteriormente se puede responder esta pregunta.

##### Variaremos los parámetros para ver que comportamiento tienen los agentes en diferentes situaciones y para responder la siguiente pregunta: ¿Qué ocurre cuando el Project Manager ajusta sus decisiones con base en el desempeño pasado del equipo?

Para esto usaremos un proyecto largo donde el PM pueda conocer el equipo y ajustarse a él. Además el PM estará generando sus propias reglas, y las usaremos en las sucesivas simulaciones para medir su efectividad ante variedades de equipos.

Equipo muy amigable : Este conjunto de parámetros aumenta la amistad entre los agentes (MIN_FRIENDSHIP y MAX_FRIENDSHIP altos), lo que puede resultar en una mayor cooperación y motivación en el equipo, mejorando el rendimiento general.

In [9]:
param_sets = []
# Parámetros de los agentes trabajadores
friedly_team = {
    MIN_PS_AGENTS : 10 ,         
    MAX_PS_AGENTS : 100,        
    MIN_FRIENDSHIP : 0.8,        # Alta amistad mínima
    MAX_FRIENDSHIP : 1.0,        # Alta amistad máxima
    MIN_MIN_ENERGY : 10 ,        
    MAX_MIN_ENERGY : 100,        
    MIN_MOTIVATION : 50 ,        # Motivación alta para trabajar
    MIN_LAZZY : 0,             
    MAX_LAZZY : 0.2,             # Baja pereza

    NUMBER_OF_WORKERS : 10
}
param_sets.append(friedly_team)

Equipo con baja amistad :
Este conjunto simula un equipo con baja amistad (MIN_FRIENDSHIP y MAX_FRIENDSHIP bajos), lo que puede dificultar la cooperación y llevar a problemas en la ejecución de tareas.

In [10]:
# Parámetros de los agentes trabajadores
not_friendly_team = {
    MIN_PS_AGENTS : 10 ,         
    MAX_PS_AGENTS : 100,         
    MIN_FRIENDSHIP : 0.0,        # Baja amistad mínima
    MAX_FRIENDSHIP : 0.2,        # Baja amistad máxima
    MIN_MIN_ENERGY : 10 ,        
    MAX_MIN_ENERGY : 100,        
    MIN_MOTIVATION : 20 ,        # Motivación mínima
    MIN_LAZZY : 0.2,             
    MAX_LAZZY : 0.4,             # Alta pereza

    NUMBER_OF_WORKERS : 10
}
param_sets.append(not_friendly_team)

Equipo con baja amistad y alta preparación :
Este conjunto simula un equipo con baja amistad (MIN_FRIENDSHIP y MAX_FRIENDSHIP bajos), lo que puede dificultar la cooperación y llevar a problemas en la ejecución de tareas, sin embargo por su alta preparación sería bueno una estrategia de reasignación de tareas. 

In [11]:
# Parámetros de los agentes trabajadores
apathic_smart = {
    MIN_PS_AGENTS : 70 ,         
    MAX_PS_AGENTS : 100,         
    MIN_FRIENDSHIP : 0.0,        # Baja amistad mínima
    MAX_FRIENDSHIP : 0.2,        # Baja amistad máxima
    MIN_MIN_ENERGY : 10 ,        
    MAX_MIN_ENERGY : 100,        
    MIN_MOTIVATION : 20 ,        # Motivación mínima
    MIN_LAZZY : 0.2,             
    MAX_LAZZY : 0.6,             # Alta pereza

    NUMBER_OF_WORKERS : 10
}
param_sets.append(apathic_smart)

Equipo con alta motivación y baja pereza : Este conjunto simula un equipo extremadamente motivado y con poca pereza, lo que debería conducir a una alta productividad.

In [12]:
# Parámetros de los agentes trabajadores
hight_motivation = {
    MIN_PS_AGENTS : 30 ,         
    MAX_PS_AGENTS : 90 ,         
    MIN_FRIENDSHIP : 0.6,        
    MAX_FRIENDSHIP : 1.0,        
    MIN_MIN_ENERGY : 40 ,        
    MAX_MIN_ENERGY : 90 ,        
    MIN_MOTIVATION : 50 ,        # Alta motivación
    MIN_LAZZY : 0.0,             # Baja pereza
    MAX_LAZZY : 0.1,             

    NUMBER_OF_WORKERS : 10
}
param_sets.append(hight_motivation)

##### Definimos la simulación

In [13]:
rules = pm.beliefs['rules']
act_rules = pm.beliefs['active_rules']
sim = []

for param in param_sets:
    agents = []
    random.seed(42)
    for i in range(param[NUMBER_OF_WORKERS]):
        agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(param[MIN_PS_AGENTS],param[MAX_PS_AGENTS]), friendship=random.uniform(param[MIN_FRIENDSHIP],param[MAX_FRIENDSHIP]))
        agents.append(agent)

    pm_ = deepcopy(pm)
    pm_.generate_rules = True
    pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents])
    # Le asignamos las reglas generadas en la simulacion pasada
    pm_.beliefs['rules'] = rules
    pm_.beliefs['active_rules'] = act_rules

    project_ = deepcopy(long_project)
    sim.append(simulator.StartSimulation(1000,agents,pm_,project_))
    # Guardamos las reglas para la proxima simulacion
    rules = pm_.beliefs['rules']
    act_rules = pm_.beliefs['active_rules']
    
# Una vez generamos diferentes reglas con diferentes equipos, ejecutemos simulaciones para comprobar su efectividad

for _ in range(10):
    for param in param_sets:
        agents = []
        for i in range(param[NUMBER_OF_WORKERS]):
            agent = WorkerAgent(id='Worker ' + str(i), problem_solving=random.randint(param[MIN_PS_AGENTS],param[MAX_PS_AGENTS]), friendship=random.uniform(param[MIN_FRIENDSHIP],param[MAX_FRIENDSHIP]))
            agents.append(agent)

        pm_ = deepcopy(pm)
        pm_.generate_rules = False
        pm_.know_workers([(agent.id,agent.problem_solving) for agent in agents])
        # Le asignamos las reglas generadas en la simulacion pasada
        pm_.beliefs['rules'] = rules
        pm_.beliefs['active_rules'] = act_rules

        project_ = deepcopy(long_project)
        sim.append(simulator.StartSimulation(1000,agents,pm_,project_))
        # Guardamos las reglas para la proxima simulacion
        rules = pm_.beliefs['rules']
        act_rules = pm_.beliefs['active_rules']


```json
conditions = ['medium team', 'highly prepared team', 'highly motivated team', 'moderately collaborative team', 'highly working team', 'few tasks completed', 'few problems', 'less than 30 percent of the rewards earned', 'less than 30 percent of project time']
desires = ['assign', 'cooperate', 'work', 'ask_report', 'priority_of_tasks', 'rewards', 'optimize_resources', 'on_time', 'avoid_risks', 'get_chances']
```
```json
conditions = ['medium team', 'highly prepared team', 'highly motivated team', 'moderately collaborative team', 'highly working team', 'few tasks completed', 'few problems', 'less than 30 percent of the rewards earned', 'less than 30 percent of project time']
desires = ['assign', 'cooperate', 'work', 'ask_report', 'priority_of_tasks', 'rewards', 'optimize_resources', 'on_time', 'avoid_risks', 'get_chances']
```
```json
conditions = ['medium team', 'highly prepared team', 'highly motivated team', 'moderately collaborative team', 'not very hardworking team', 'few tas

In [16]:
for simulation in simulations:
    simulation.save_log_to_csv()

for s in sim:
    s.save_log_to_csv()